In [39]:
def parse_data_line(s):
    addr = int(s[:5])
    com_obj = int(s[17:26])
    name = s[26:41].strip()
    dim = s[41:46].strip()
    data_type = s[46:58].strip()
    data_len = int(s[58:60].strip())
    dec = int(s[60:63].strip())
    data_min = int(s[63:70].strip())
    data_max = int(s[70:77].strip())
    group = s[77:].strip()
    return addr, com_obj, name, dim, data_type, data_len, dec, data_min, data_max, group

In [1]:
import time

start_time = time.time()

def parse_data_line(s):
    addr = int(s[:5])-40001
    com_obj = int(s[17:26])
    name = s[26:41].strip()
    dim = s[41:46].strip()
    data_type = s[46:58].strip()
    data_len = int(s[58:60].strip())
    dec = s[60:63].strip()
    data_min = s[63:70].strip()
    data_max = s[70:77].strip()
    group = s[77:].strip()
    return addr, com_obj, name, dim, data_type, data_len, dec, data_min, data_max, group

def read_mb_register(i, reg):
    while 1:
        try:
            return i.read_register(reg, signed=True)
        except Exception as e:
            continue
                
def read_mb_registers_long(g, reg):
    for i in doc:
        while 1:
            try:
                if i.address == 1 or i.address == 2 or i.address == 1:
                    print(str(i.read_long(reg)))
                elif i.address == 3 or i.address == 4 or i.address == 5:
                    print(i.read_long(reg))
                break
            except:
                continue

with open('./mb_table/doc_3516_mb.txt', encoding='cp1252') as f:
    data_lines = f.readlines()
    
data_dict = {}
count = 0

for i in data_lines[2:]:
    if i[0] == '\n': # Если пустая строка, то заканчиваем опрос
        break
    elif i[0] == '4': # Если начинается с 4-х, то
        line = parse_data_line(i)
        if line[5]<3:
            count += 1
            data = read_mb_register(g1, line[0])
            if line[6] != '-' and line[6] != '0':
                data = data/10**int(line[6])
            print(f'{count} {line[2]}: \t {data} {line[3]}')

# for addr in data_arr:
#     count += 1
#     if isinstance(addr, int):
#         print(addr, read_mb_register(g1, addr))
#     else:
#         pass
#         if int(addr.strip()[-2]) != 2:
#             a = int(addr.split('-')[0] - 40001)
#             print(addr, 'long', '\t', read_mb_registers_long(g1, addr))
        
print((f'{time.time() - start_time:.1f} seconds'))

NameError: name 'g1' is not defined

In [70]:
start_time = time.time()

def read_mb_register(i, reg):
    while 1:
        try:
            return i.read_register(reg)
        except Exception as e:
            continue
            
def get_protect(genset):
    level = ['inactive', 'N/A', 'active, confirmed', 'active, but blocked or delay still running', 'previously active, not confirmed yet', 'N/A','active, not confirmed yet', 'active, not confirmed yet, blocked']
    sensfail = ['Sensor failure not active', 'Sensor failure active, confirmed', 'Sensor failure previously active, not confirmed yet', 'Sensor failure active, not confirmed yet']
    
    if genset.address == 1 or genset.address == 2:
        with open('protections_3516.txt') as f:
            data_lines = f.readlines()
    elif genset.address == 3 or genset.address == 4 or genset.address == 5:
        with open('protections_3520.txt') as f:
            data_lines = f.readlines()
        
    protections = []
    ret = []
    
    for line in data_lines:
        # [6211, 'Bus V L2-L3', 'Bus V L3-L1']
        protections.append((int(line[:5])-40001, line[17:37].strip(), line[37:].strip()))
        
    for p in protections:
        addr, protect2, protect1 = p
        protection = read_mb_register(genset, addr)
    
        prot1_level1 = 0b0000000000000111 & protection
        prot1_level2 = (0b0000000000111000 & protection)>>3
        prot1_sens = (0b0000000011000000 & protection)>>6
        prot2_level1 = 0b0000011100000000 & protection>>8
        prot2_level2 = (0b0011100000000000 & protection)>>11
        prot2_sens = (0b1100000000000000 & protection)>>14

        if prot1_level1: ret.append(f'{protect1}. Level 1: {level[prot1_level1]}')
        if prot1_level2: ret.append(f'{protect1}. Level 2: {level[prot1_level2]}')
        if prot1_sens: ret.append(f'{protect1}. Sensor failure: {sensfail[prot1_sens]}')
        if prot2_level1: ret.append(f'{protect2}. Level 1: {level[prot2_level1]}')
        if prot2_level2: ret.append(f'{protect2}. Level 2: {level[prot2_level2]}')
        if prot2_sens: ret.append(f'{protect2}. Sensor failure: {sensfail[prot2_sens]}')
        
    return ret

for i in get_protect(g5):
    print(i)
    
print((f'{time.time() - start_time:.1f} seconds'))

KeyboardInterrupt: 

In [5]:
import logging
logging.DEBUG?

In [ ]:
import telebot
import minimalmodbus 
import time
import logging

logger = logging.getLogger('engine_state')
logger.setLevel(logging.DEBUG)

g1 = minimalmodbus.Instrument('/dev/ttyS0', 1)
g1.serial.baudrate = 19200

g2 = minimalmodbus.Instrument('/dev/ttyS0', 2)
g2.serial.baudrate = 19200

g3 = minimalmodbus.Instrument('/dev/ttyS0', 3)
g3.serial.baudrate = 19200

g4 = minimalmodbus.Instrument('/dev/ttyS0', 4)
g4.serial.baudrate = 19200

g5 = minimalmodbus.Instrument('/dev/ttyS0', 5)
g5.serial.baudrate = 19200

token = '1325955552:AAHQJzvhu-IhcMpsVGX5dUsfXTJ7fFECoJs'
tb = telebot.TeleBot(token)

gensets = ('ГПГУ 1', 'ГПГУ 2', 'ГПГУ 3', 'ГПГУ 4', 'ГПГУ 5')
doc = (g1, g2, g3, g4, g5)

def read_mb_register(i, reg):
    logger.debug(f'read_mb_register({i.address}, {reg})')
    for c in range(10):
        try:
            return i.read_register(reg)
        except Exception as e:
            continue
    return -1

def get_engines_state():
    logger.debug(f'get_engines_state()')
    engine_state_3516 = ('Init', 'Ready', 'NotReady', 'Prestart', 'Cranking', 'Pause', 'Starting', 'Running', 'Loaded', 'Soft unld', 'Cooling', 'Stop', 'Shutdown', 'Ventil', 'EmergMan', 'Cooldown', 'Offload', 'Soft load', 'WaitStop', 'Warming', 'SDVentil', None)
    engine_state_3520 = ('Init', 'Ready', 'NotReady', 'Prestart', 'Cranking', 'Pause', 'Starting', 'Running', 'Loaded', 'Soft unld', 'Cooling', 'Stop', 'Shutdown', 'Ventil', 'EmergMan', 'Cooldown', 'Offload', 'Soft load', 'WaitStop', 'Warming', 'SDVentil', 'WD test', 'GasVTest', 'StrtCndWai', None)
    return [engine_state_3516[read_mb_register(g1, 162)], engine_state_3516[read_mb_register(g2, 162)], engine_state_3520[read_mb_register(g3, 295)], engine_state_3520[read_mb_register(g4, 295)], engine_state_3520[read_mb_register(g5, 295)]]

def get_breakers_state():
    logger.debug(f'get_breakers_state()')
    breaker_state = ('Init', 'BrksOff', 'IslOper', 'MainsOper', 'ParalOper', 'RevSync', 'Synchro', 'MainsFlt', 'ValidFlt', 'MainsRet', 'MultIslOp', 'MultParOp', 'EmergMan', 'StrUpSync', None)
    return [breaker_state[read_mb_register(g1, 163)], breaker_state[read_mb_register(g2, 163)], breaker_state[read_mb_register(g3, 296)], breaker_state[read_mb_register(g4, 296)], breaker_state[read_mb_register(g5, 296)]]

def get_protect(genset):
    logger.debug(f'get_protect({genset.address})')
    level = ['inactive', 'N/A', 'active, confirmed', 'active, but blocked or delay still running', 'previously active, not confirmed yet', 'N/A','active, not confirmed yet', 'active, not confirmed yet, blocked']
    sensfail = ['Sensor failure not active', 'Sensor failure active, confirmed', 'Sensor failure previously active, not confirmed yet', 'Sensor failure active, not confirmed yet']
    
    if genset.address == 1 or genset.address == 2:
        with open('protections_3516.txt') as f:
            data_lines = f.readlines()
    elif genset.address == 3 or genset.address == 4 or genset.address == 5:
        with open('protections_3520.txt') as f:
            data_lines = f.readlines()
        
    protections = []
    ret = []
    
    for line in data_lines:
        # [6211, 'Bus V L2-L3', 'Bus V L3-L1']
        protections.append((int(line[:5])-40001, line[17:37].strip(), line[37:].strip()))
        
    for p in protections:
        addr, protect2, protect1 = p
        protection = read_mb_register(genset, addr)
    
        prot1_level1 = 0b0000000000000111 & protection
        prot1_level2 = (0b0000000000111000 & protection)>>3
        prot1_sens = (0b0000000011000000 & protection)>>6
        prot2_level1 = 0b0000011100000000 & protection>>8
        prot2_level2 = (0b0011100000000000 & protection)>>11
        prot2_sens = (0b1100000000000000 & protection)>>14
        
        if prot1_level1: ret.append(f'{protect1}. Level 1: {level[prot1_level1]}\n')
        if prot1_level2: ret.append(f'{protect1}. Level 2: {level[prot1_level2]}\n')
        if prot1_sens: ret.append(f'{protect1}. Sensor failure: {sensfail[prot1_sens]}\n')
        if prot2_level1: ret.append(f'{protect2}. Level 1: {level[prot2_level1]}\n')
        if prot2_level2: ret.append(f'{protect2}. Level 2: {level[prot2_level2]}\n')
        if prot2_sens: ret.append(f'{protect2}. Sensor failure: {sensfail[prot2_sens]}\n')
        
    return ret

es_old = get_engines_state()
bs_old = get_breakers_state()

# Основной цикл
while 1:
    logger.debug(f'Начало цикла')
    es_new = get_engines_state()
    bs_new = get_breakers_state()
    for i in range(5):
        # Читаю состояния двигателей
        if es_new[i] == None: es_new[i] = es_old[i] 
        if es_new[i] != es_old[i]:
            logger.debug(f'Новое значение engine status {es_new[i]}')
            es_old[i] = es_new[i]
            tb.send_message(723253749, f'{gensets[i]}. Engine state: {es_new[i]}')
            if es_new[i] in (2, 9, 12, 16, 20):
                msg = ''
                for i in get_protect(doc(i)):
                    msg += i
                if msg: tb.send_message(723253749, msg)
        # Читаю состояния выключателей
        if bs_new[i] == None: bs_new[i] = bs_old[i] 
        if bs_new[i] != bs_old[i]:
            logger.debug(f'Новое значение engine status {bs_new[i]}')
            bs_old[i] = bs_new[i]
            tb.send_message(723253749, f'{gensets[i]}. Breaker state: {bs_new[i]}')
        
    time.sleep(1)

DEBUG:engine_state:get_engines_state()
DEBUG:engine_state:read_mb_register(1, 162)
DEBUG:engine_state:read_mb_register(2, 162)
DEBUG:engine_state:read_mb_register(3, 295)
DEBUG:engine_state:read_mb_register(4, 295)
DEBUG:engine_state:read_mb_register(5, 295)
DEBUG:engine_state:get_breakers_state()
DEBUG:engine_state:read_mb_register(1, 163)
DEBUG:engine_state:read_mb_register(2, 163)
DEBUG:engine_state:read_mb_register(3, 296)
DEBUG:engine_state:read_mb_register(4, 296)
DEBUG:engine_state:read_mb_register(5, 296)
DEBUG:engine_state:Начало цикла
DEBUG:engine_state:get_engines_state()
DEBUG:engine_state:read_mb_register(1, 162)
DEBUG:engine_state:read_mb_register(2, 162)
DEBUG:engine_state:read_mb_register(3, 295)
DEBUG:engine_state:read_mb_register(4, 295)
DEBUG:engine_state:read_mb_register(5, 295)
DEBUG:engine_state:get_breakers_state()
DEBUG:engine_state:read_mb_register(1, 163)
DEBUG:engine_state:read_mb_register(2, 163)
DEBUG:engine_state:read_mb_register(3, 296)
DEBUG:engine_state

DEBUG:engine_state:Начало цикла
DEBUG:engine_state:get_engines_state()
DEBUG:engine_state:read_mb_register(1, 162)
DEBUG:engine_state:read_mb_register(2, 162)
DEBUG:engine_state:read_mb_register(3, 295)
DEBUG:engine_state:read_mb_register(4, 295)
DEBUG:engine_state:read_mb_register(5, 295)
DEBUG:engine_state:get_breakers_state()
DEBUG:engine_state:read_mb_register(1, 163)
DEBUG:engine_state:read_mb_register(2, 163)
DEBUG:engine_state:read_mb_register(3, 296)
DEBUG:engine_state:read_mb_register(4, 296)
DEBUG:engine_state:read_mb_register(5, 296)
DEBUG:engine_state:Начало цикла
DEBUG:engine_state:get_engines_state()
DEBUG:engine_state:read_mb_register(1, 162)
DEBUG:engine_state:read_mb_register(2, 162)
DEBUG:engine_state:read_mb_register(3, 295)
DEBUG:engine_state:read_mb_register(4, 295)
DEBUG:engine_state:read_mb_register(5, 295)
DEBUG:engine_state:get_breakers_state()
DEBUG:engine_state:read_mb_register(1, 163)
DEBUG:engine_state:read_mb_register(2, 163)
DEBUG:engine_state:read_mb_reg

DEBUG:engine_state:read_mb_register(5, 296)
DEBUG:engine_state:Начало цикла
DEBUG:engine_state:get_engines_state()
DEBUG:engine_state:read_mb_register(1, 162)
DEBUG:engine_state:read_mb_register(2, 162)
DEBUG:engine_state:read_mb_register(3, 295)
DEBUG:engine_state:read_mb_register(4, 295)
DEBUG:engine_state:read_mb_register(5, 295)
DEBUG:engine_state:get_breakers_state()
DEBUG:engine_state:read_mb_register(1, 163)
DEBUG:engine_state:read_mb_register(2, 163)
DEBUG:engine_state:read_mb_register(3, 296)
DEBUG:engine_state:read_mb_register(4, 296)
DEBUG:engine_state:read_mb_register(5, 296)
DEBUG:engine_state:Начало цикла
DEBUG:engine_state:get_engines_state()
DEBUG:engine_state:read_mb_register(1, 162)
DEBUG:engine_state:read_mb_register(2, 162)
DEBUG:engine_state:read_mb_register(3, 295)
DEBUG:engine_state:read_mb_register(4, 295)
DEBUG:engine_state:read_mb_register(5, 295)
DEBUG:engine_state:get_breakers_state()
DEBUG:engine_state:read_mb_register(1, 163)
DEBUG:engine_state:read_mb_reg

DEBUG:engine_state:read_mb_register(4, 296)
DEBUG:engine_state:read_mb_register(5, 296)
DEBUG:engine_state:Начало цикла
DEBUG:engine_state:get_engines_state()
DEBUG:engine_state:read_mb_register(1, 162)
DEBUG:engine_state:read_mb_register(2, 162)
DEBUG:engine_state:read_mb_register(3, 295)
DEBUG:engine_state:read_mb_register(4, 295)
DEBUG:engine_state:read_mb_register(5, 295)
DEBUG:engine_state:get_breakers_state()
DEBUG:engine_state:read_mb_register(1, 163)
DEBUG:engine_state:read_mb_register(2, 163)
DEBUG:engine_state:read_mb_register(3, 296)
DEBUG:engine_state:read_mb_register(4, 296)
DEBUG:engine_state:read_mb_register(5, 296)
DEBUG:engine_state:Начало цикла
DEBUG:engine_state:get_engines_state()
DEBUG:engine_state:read_mb_register(1, 162)
DEBUG:engine_state:read_mb_register(2, 162)
DEBUG:engine_state:read_mb_register(3, 295)
DEBUG:engine_state:read_mb_register(4, 295)
DEBUG:engine_state:read_mb_register(5, 295)
DEBUG:engine_state:get_breakers_state()
DEBUG:engine_state:read_mb_reg

DEBUG:engine_state:read_mb_register(3, 296)
DEBUG:engine_state:read_mb_register(4, 296)
DEBUG:engine_state:read_mb_register(5, 296)
DEBUG:engine_state:Начало цикла
DEBUG:engine_state:get_engines_state()
DEBUG:engine_state:read_mb_register(1, 162)
DEBUG:engine_state:read_mb_register(2, 162)
DEBUG:engine_state:read_mb_register(3, 295)
DEBUG:engine_state:read_mb_register(4, 295)
DEBUG:engine_state:read_mb_register(5, 295)
DEBUG:engine_state:get_breakers_state()
DEBUG:engine_state:read_mb_register(1, 163)
DEBUG:engine_state:read_mb_register(2, 163)
DEBUG:engine_state:read_mb_register(3, 296)
DEBUG:engine_state:read_mb_register(4, 296)
DEBUG:engine_state:read_mb_register(5, 296)
DEBUG:engine_state:Начало цикла
DEBUG:engine_state:get_engines_state()
DEBUG:engine_state:read_mb_register(1, 162)
DEBUG:engine_state:read_mb_register(2, 162)
DEBUG:engine_state:read_mb_register(3, 295)
DEBUG:engine_state:read_mb_register(4, 295)
DEBUG:engine_state:read_mb_register(5, 295)
DEBUG:engine_state:get_bre

...

DEBUG:engine_state:get_breakers_state()
DEBUG:engine_state:read_mb_register(1, 163)
DEBUG:engine_state:read_mb_register(2, 163)
DEBUG:engine_state:read_mb_register(3, 296)
DEBUG:engine_state:read_mb_register(4, 296)
DEBUG:engine_state:read_mb_register(5, 296)
DEBUG:engine_state:Начало цикла
DEBUG:engine_state:get_engines_state()
DEBUG:engine_state:read_mb_register(1, 162)
DEBUG:engine_state:read_mb_register(2, 162)
DEBUG:engine_state:read_mb_register(3, 295)
DEBUG:engine_state:read_mb_register(4, 295)
DEBUG:engine_state:read_mb_register(5, 295)
DEBUG:engine_state:get_breakers_state()
DEBUG:engine_state:read_mb_register(1, 163)
DEBUG:engine_state:read_mb_register(2, 163)
DEBUG:engine_state:read_mb_register(3, 296)
DEBUG:engine_state:read_mb_register(4, 296)
DEBUG:engine_state:read_mb_register(5, 296)
DEBUG:engine_state:Начало цикла
DEBUG:engine_state:get_engines_state()
DEBUG:engine_state:read_mb_register(1, 162)
DEBUG:engine_state:read_mb_register(2, 162)
DEBUG:engine_state:read_mb_reg

DEBUG:engine_state:read_mb_register(5, 295)
DEBUG:engine_state:get_breakers_state()
DEBUG:engine_state:read_mb_register(1, 163)
DEBUG:engine_state:read_mb_register(2, 163)
DEBUG:engine_state:read_mb_register(3, 296)
DEBUG:engine_state:read_mb_register(4, 296)
DEBUG:engine_state:read_mb_register(5, 296)
DEBUG:engine_state:Начало цикла
DEBUG:engine_state:get_engines_state()
DEBUG:engine_state:read_mb_register(1, 162)
DEBUG:engine_state:read_mb_register(2, 162)
DEBUG:engine_state:read_mb_register(3, 295)
DEBUG:engine_state:read_mb_register(4, 295)
DEBUG:engine_state:read_mb_register(5, 295)
DEBUG:engine_state:get_breakers_state()
DEBUG:engine_state:read_mb_register(1, 163)
DEBUG:engine_state:read_mb_register(2, 163)
DEBUG:engine_state:read_mb_register(3, 296)
DEBUG:engine_state:read_mb_register(4, 296)
DEBUG:engine_state:read_mb_register(5, 296)
DEBUG:engine_state:Начало цикла
DEBUG:engine_state:get_engines_state()
DEBUG:engine_state:read_mb_register(1, 162)
DEBUG:engine_state:read_mb_reg

DEBUG:engine_state:read_mb_register(4, 295)
DEBUG:engine_state:read_mb_register(5, 295)
DEBUG:engine_state:get_breakers_state()
DEBUG:engine_state:read_mb_register(1, 163)
DEBUG:engine_state:read_mb_register(2, 163)
DEBUG:engine_state:read_mb_register(3, 296)
DEBUG:engine_state:read_mb_register(4, 296)
DEBUG:engine_state:read_mb_register(5, 296)
DEBUG:engine_state:Начало цикла
DEBUG:engine_state:get_engines_state()
DEBUG:engine_state:read_mb_register(1, 162)
DEBUG:engine_state:read_mb_register(2, 162)
DEBUG:engine_state:read_mb_register(3, 295)
DEBUG:engine_state:read_mb_register(4, 295)
DEBUG:engine_state:read_mb_register(5, 295)
DEBUG:engine_state:get_breakers_state()
DEBUG:engine_state:read_mb_register(1, 163)
DEBUG:engine_state:read_mb_register(2, 163)
DEBUG:engine_state:read_mb_register(3, 296)
DEBUG:engine_state:read_mb_register(4, 296)
DEBUG:engine_state:read_mb_register(5, 296)
DEBUG:engine_state:Начало цикла
DEBUG:engine_state:get_engines_state()
DEBUG:engine_state:read_mb_reg

DEBUG:engine_state:read_mb_register(3, 295)
DEBUG:engine_state:read_mb_register(4, 295)
DEBUG:engine_state:read_mb_register(5, 295)
DEBUG:engine_state:get_breakers_state()
DEBUG:engine_state:read_mb_register(1, 163)
DEBUG:engine_state:read_mb_register(2, 163)
DEBUG:engine_state:read_mb_register(3, 296)
DEBUG:engine_state:read_mb_register(4, 296)
DEBUG:engine_state:read_mb_register(5, 296)
DEBUG:engine_state:Начало цикла
DEBUG:engine_state:get_engines_state()
DEBUG:engine_state:read_mb_register(1, 162)
DEBUG:engine_state:read_mb_register(2, 162)
DEBUG:engine_state:read_mb_register(3, 295)
DEBUG:engine_state:read_mb_register(4, 295)
DEBUG:engine_state:read_mb_register(5, 295)
DEBUG:engine_state:get_breakers_state()
DEBUG:engine_state:read_mb_register(1, 163)
DEBUG:engine_state:read_mb_register(2, 163)
DEBUG:engine_state:read_mb_register(3, 296)
DEBUG:engine_state:read_mb_register(4, 296)
DEBUG:engine_state:read_mb_register(5, 296)
DEBUG:engine_state:Начало цикла
DEBUG:engine_state:get_en

DEBUG:engine_state:read_mb_register(2, 162)
DEBUG:engine_state:read_mb_register(3, 295)
DEBUG:engine_state:read_mb_register(4, 295)
DEBUG:engine_state:read_mb_register(5, 295)
DEBUG:engine_state:get_breakers_state()
DEBUG:engine_state:read_mb_register(1, 163)
DEBUG:engine_state:read_mb_register(2, 163)
DEBUG:engine_state:read_mb_register(3, 296)
DEBUG:engine_state:read_mb_register(4, 296)
DEBUG:engine_state:read_mb_register(5, 296)
DEBUG:engine_state:Начало цикла
DEBUG:engine_state:get_engines_state()
DEBUG:engine_state:read_mb_register(1, 162)
DEBUG:engine_state:read_mb_register(2, 162)
DEBUG:engine_state:read_mb_register(3, 295)
DEBUG:engine_state:read_mb_register(4, 295)
DEBUG:engine_state:read_mb_register(5, 295)
DEBUG:engine_state:get_breakers_state()
DEBUG:engine_state:read_mb_register(1, 163)
DEBUG:engine_state:read_mb_register(2, 163)
DEBUG:engine_state:read_mb_register(3, 296)
DEBUG:engine_state:read_mb_register(4, 296)
DEBUG:engine_state:read_mb_register(5, 296)
DEBUG:engine_

In [14]:
read_mb_register(g5, 5750)

DEBUG:engine_state:read_mb_register(5, 5750)


-1

In [33]:
b1 = g2.read_register(2)

bin1 = ['RemStart/Stop',
'Emergency stop',
'GCB Closed',
'GCB Open',
'GCB Fail',
'GenMess QF2SHS',
'BusMess QF3SHS',
'FireAlarm',
'CO/CH4Lev1',
'CO/CH4Lev2',
'RFan M1-M3 On',
'RFan M1 Fault',
'RFan M2 Fault',
'QF17,18,19 OFF',
'not used',
'RFan M3 Fault']

mask = 1

for i in range(16):
    flag = 0
    if b1 & mask: flag = 1
    print(f'{bin1[i]} \t {flag}')
    mask = mask << 1

RemStart/Stop 	 0
Emergency stop 	 1
GCB Closed 	 1
GCB Open 	 0
GCB Fail 	 0
GenMess QF2SHS 	 1
BusMess QF3SHS 	 1
FireAlarm 	 0
CO/CH4Lev1 	 0
CO/CH4Lev2 	 0
RFan M1-M3 On 	 1
RFan M1 Fault 	 0
RFan M2 Fault 	 0
QF17,18,19 OFF 	 0
not used 	 0
RFan M3 Fault 	 0


In [2]:
import minimalmodbus, time

g1 = minimalmodbus.Instrument('/dev/ttyS0', 1)
g1.serial.baudrate = 19200

g2 = minimalmodbus.Instrument('/dev/ttyS0', 2)
g2.serial.baudrate = 19200

g3 = minimalmodbus.Instrument('/dev/ttyS0', 3)
g3.serial.baudrate = 19200

g4 = minimalmodbus.Instrument('/dev/ttyS0', 4)
g4.serial.baudrate = 19200

g5 = minimalmodbus.Instrument('/dev/ttyS0', 5)
g5.serial.baudrate = 19200

doc = [g1, g2, g3, g4, g5]